## Meteo Bakery - Combine datasets
This notebook serves to combine df_full data with the weather summary statistics.

### import libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from itertools import chain

### load data

In [ ]:
# load df_full data
sales = pd.read_excel('../data/neueFische_Umsaetze_Baeckerei.xlsx')

In [ ]:
# load data on engineered weather features
weather_stats = pd.read_csv('../data/summary_stats.csv')

In [ ]:
# load holidays data

# school holidays from https://www.schulferien.org/oesterreich/ferien/2012/
school_hols = pd.read_excel("../data/school_holidays.xlsx")

# public holidays from google search "Feiertage Wien 'YEAR'"
public_hols = pd.read_excel("../data/public_holidays.xlsx")
public_hols.date = pd.to_datetime(public_hols.date)

In [ ]:
# load Corona data
corona = pd.read_excel("../data/corona-measures-vienna.xlsx")

### Feature Engineering - Sales

In [ ]:
# get basic information on datatypes and missings
sales.info()

In [ ]:
# generate location column based on branch
# Filiale 1: U-Bahn
# Filiale 2: Innenstadt
# Filiale 3: Bahnhof

sales['Branch'] = sales.Branch.apply(lambda x: 'Metro' if x==1 else 'Center' if x==2 else 'Train_Station')
sales.head()

There are three missing values in the sales data ('SoldTurnver').

In [ ]:
sales.columns

In [ ]:
# rename columns
sales.rename(columns={'Branch': 'branch', 'PredictionGroupName': 'product', 'SoldTurnver': 'turnover'}, inplace=True)

In [ ]:
sales.rename(columns={'Date': 'date'}, inplace=True)
sales.head()

In [ ]:
#relabel products
sales['product'] = sales['product'].map({'Mischbrote':'Brown Bread',
                                'Weizenbrötchen':'Wheat Rolls',
                                'klassischer Kuchen':'Cakes',
                                'handliches Gebäck':'Pastries',
                                'herzhafter Snack':'Savoury Snack'})

In [ ]:
# count dates per branch and product category
sales.groupby(['branch', 'product'])['date'].count()

As can be seen, not all dates are equally represented per group. This indicates that dates are not continuously progressing, but that there gaps present in the dates. Thus, there must be missing dates. Indeed, the first Covid19 lockdown has already removed from the data, representing one of possibly more gaps.

### Generate a time series of consecutive dates as backbone
To avoid such gaps, we will first generate a datetime column with consecutive gaps starting and ending with the first and last registered date. The other data will then be merged into that continuous date column, with gaps in certain columns being filled up with NaNs. These NaNs can be handled strategically during later analysis and modeling steps.

In [ ]:
consec_dates = pd.DataFrame({'date':pd.date_range(sales.date.min(), sales.date.max())})

In [ ]:
print(sales.date.nunique())
print(consec_dates.date.nunique())

In [ ]:
consec_dates.date.nunique() * 15

### repeat the dates for each branch and product category

In [ ]:
consec_dates[['Metro', 'Center', 'Train_Station']] = 'Metro', 'Center', 'Train_Station'

In [ ]:
consec_dates.set_index('date', inplace=True)
consec_dates.head()

In [ ]:
consec_dates = consec_dates.stack().reset_index(name='branch').drop(columns=['level_1'])
consec_dates.head()

In [ ]:
products = sales['product'].unique()
consec_dates[products] = products

In [ ]:
consec_dates.set_index(['date', 'branch'], inplace=True)
consec_dates.head()

In [ ]:
consec_dates = consec_dates.stack().reset_index(name='product').drop(columns=['level_2'])

In [ ]:
consec_dates.head()

In [ ]:
consec_dates.shape

### Merge dataframes

#### merge sales into backbone

In [ ]:
df_full = consec_dates.merge(sales, on=['date', 'branch', 'product'], how='left')

In [ ]:
df_full.head()

In [ ]:
df_full.shape

In [ ]:
df_full.groupby(['branch', 'product'])['date'].count()

In [ ]:
df_full.date.value_counts()

### append additional time information

In [ ]:
# extract time features from Date column
df_full['year'] = df_full.date.dt.year
df_full['month'] = df_full.date.dt.month
df_full['week'] = df_full.date.dt.week
df_full['day_of_month'] = df_full.date.dt.day
df_full['day_of_week'] = df_full.date.dt.dayofweek

#### append holiday and Covid information

In [ ]:
# append holidays by creating true/false columns
df_full["school_holiday"] = df_full["date"].isin(school_hols["date"])
df_full["public_holiday"] = df_full["date"].isin(public_hols["date"])

# public holiday name
df_full["p_hol_name"] = df_full["public_holiday"].copy()
for x in range(public_hols.shape[0]):
    df_full.loc[df_full["date"] == public_hols.iloc[x, 0], "p_hol_name"] = public_hols.iloc[x, 1]
# school holiday name
df_full["s_hol_name"] = df_full["school_holiday"].copy()
for x in range(school_hols.shape[0]):
    df_full.loc[df_full["date"] == school_hols.iloc[x, 0], "s_hol_name"] = school_hols.iloc[x, 1]


In [ ]:
# broadcast lockdown times
df_full["lock"] = 'open'
df_full.loc[(df_full.date >= pd.to_datetime("2020-03-10")) & (df_full.date < pd.to_datetime("2020-04-14")),"lock"] = "lockdown"
df_full.loc[(df_full.date >= pd.to_datetime("2020-11-03")) & (df_full.date < pd.to_datetime("2020-11-17")),"lock"] = "lockdown_light"
df_full.loc[(df_full.date >= pd.to_datetime("2020-11-17")) & (df_full.date < pd.to_datetime("2020-12-06")),"lock"] = "lockdown"
df_full.loc[(df_full.date >= pd.to_datetime("2020-12-26")) & (df_full.date < pd.to_datetime("2021-02-07")),"lock"] = "lockdown"
df_full.loc[(df_full.date >= pd.to_datetime("2021-04-01")) & (df_full.date < pd.to_datetime("2021-05-02")),"lock"] = "lockdown"
df_full.loc[(df_full.date >= pd.to_datetime("2021-11-08")) & (df_full.date < pd.to_datetime("2021-12-31")),"lock"] = "lockdown"

# Deal with abrupt shifts in sales data

In [ ]:
df_full["product"]

In [ ]:
# for example, "Savoury Snack" in "Metro" has a sudden drop in sales after 2014 which is likely due to e.g. a change in categories or decrease in store size or something else which cannot be accounted for by a model
plt.figure(figsize = (20,10))
sns.scatterplot(data =df_full[(df_full["branch"] == "Metro") & (df_full["product"] == "Savoury Snack")], x = "date", y = "turnover")
plt.xticks(rotation=45, horizontalalignment="right");

In [ ]:

def mean_shift(df_full, branches, products, date1, date2, date3, date4, date5, date6):
    
    for br in branches:
        for pr in products:
            # multiplying with the respective mean shift compared to timeframe between date1 and date2 and date3 and date4
            mean_before = df_full["turnover"][(df_full["product"] == pr) & (df_full["branch"] == br) &
            (((df_full["date"] >= date1) & 
            (df_full["date"] < date2)) | ((df_full["date"] >= date3) & 
            (df_full["date"] < date4)))].mean()
            mean_of_interest = df_full["turnover"][(df_full["product"] == pr) & (df_full["branch"] == br) &
            (df_full["date"] >= date5) & 
            (df_full["date"] < date6)].mean()
            df_full["turnover"][(df_full["product"] == pr) & (df_full["branch"] == br) &
            (df_full["date"] >= date5) & 
            (df_full["date"] < date6)] = df_full["turnover"][(df_full["product"] == pr) & (df_full["branch"] == br) &
            (df_full["date"] >= date5) & 
            (df_full["date"] < date6)] * mean_before / mean_of_interest
    return df_full

df_full = mean_shift(df_full, branches = ["Metro", "Train_Station", "Center"], products = ["Wheat Rolls"], 
date1 = pd.to_datetime("2016-05-01"), date2 = pd.to_datetime("2016-07-30"), 
date3 = pd.to_datetime("2017-05-01"), date4 = pd.to_datetime("2017-07-30"),
date5 = pd.to_datetime("2018-05-01"), date6 = pd.to_datetime("2018-07-30"))

df_full = mean_shift(df_full, branches = ["Train_Station"], products = ["Wheat Rolls"], 
date1 = pd.to_datetime("2016-01-01"), date2 = pd.to_datetime("2016-12-31"), 
date3 = pd.to_datetime("2017-01-01"), date4 = pd.to_datetime("2017-12-31"),
date5 = pd.to_datetime("2012-07-01"), date6 = pd.to_datetime("2015-12-31"))

df_full = mean_shift(df_full, branches = ["Metro"], products = ["Brown Bread"], 
date1 = pd.to_datetime("2016-01-01"), date2 = pd.to_datetime("2016-12-31"), 
date3 = pd.to_datetime("2017-01-01"), date4 = pd.to_datetime("2017-12-31"),
date5 = pd.to_datetime("2018-08-01"), date6 = pd.to_datetime("2020-03-11"))

df_full = mean_shift(df_full, branches = ["Train_Station"], products = ["Savoury Snack"], 
date1 = pd.to_datetime("2016-01-01"), date2 = pd.to_datetime("2016-12-31"), 
date3 = pd.to_datetime("2017-01-01"), date4 = pd.to_datetime("2017-12-31"),
date5 = pd.to_datetime("2012-01-01"), date6 = pd.to_datetime("2015-09-30"))

df_full = mean_shift(df_full, branches = ["Train_Station"], products = ["Savoury Snack"], 
date1 = pd.to_datetime("2016-10-01"), date2 = pd.to_datetime("2016-12-31"), 
date3 = pd.to_datetime("2017-10-01"), date4 = pd.to_datetime("2017-12-31"),
date5 = pd.to_datetime("2015-10-01"), date6 = pd.to_datetime("2015-12-31"))

df_full = mean_shift(df_full, branches = ["Metro"], products = ["Savoury Snack"], 
date1 = pd.to_datetime("2016-01-01"), date2 = pd.to_datetime("2016-12-31"), 
date3 = pd.to_datetime("2017-01-01"), date4 = pd.to_datetime("2017-12-31"),
date5 = pd.to_datetime("2012-01-01"), date6 = pd.to_datetime("2014-12-31"))

In [ ]:
# after adjustment
g = sns.FacetGrid(data = df_full, row = "product", col = "branch", aspect = 3, 
sharex = False, sharey = False)
g.map(sns.scatterplot, "date", "turnover")
plt.savefig("../plots/sales.overview.perbranchandproduct.adjusted.png");

### merge with weather statistics

In [ ]:
weather_stats.info()

In [ ]:
# parse date to datetime
weather_stats['date'] = pd.to_datetime(weather_stats['date'])

In [ ]:
# merge dataframes
df_joined = df_full.merge(weather_stats, on='date', how='left')

In [ ]:
df_joined.head(20)

In [ ]:
# create dummies for general weather
# merging and cleaning condition column
df_joined["general_w"] = df_joined["condition_total"]
df_joined["general_w"][df_joined["general_w"] == "['rainy' 'snowy']"] = "snowy"
df_joined["general_w"][df_joined["general_w"] == "['clear' 'cloudy' 'rainy']"] = "rainy"
df_joined["general_w"][df_joined["general_w"] == "['clear' 'cloudy']"] = "cloudy"
df_joined["general_w"][df_joined["general_w"] == "['clear' 'sormy']"] = "stormy"
df_joined["general_w"][df_joined["general_w"] == "['cloudy' 'rainy']"] = "rainy"
df_joined["general_w"][df_joined["general_w"] == "['cloudy' 'snowy']"] = "snowy"
df_joined["general_w"][df_joined["general_w"] == "['cloudy' 'rainy' 'snowy']"] = "snowy"
df_joined["general_w"][df_joined["general_w"] == "['foggy' 'rainy']"] = "foggy"
df_joined["general_w"][df_joined["general_w"] == "['clear' 'cloudy' 'snowy']"] = "snowy"
df_joined["general_w"][df_joined["general_w"] == "['cloudy' 'rainy' 'stormy]"] = "stormy"
df_joined["general_w"][df_joined["general_w"] == "['clear' 'foggy']"] = "foggy"
df_joined["general_w"][df_joined["general_w"] == "['clear' 'cloudy' 'foggy']"] = "foggy"
df_joined["general_w"][df_joined["general_w"] == "['clear' 'rainy']"] = "rainy"
df_joined["general_w"][df_joined["general_w"] == "['cloudy' 'foggy' 'rainy']"] = "foggy"
df_joined["general_w"][df_joined["general_w"] == "['foggy' 'snowy']"] = "foggy"
df_joined["general_w"][df_joined["general_w"] == "['clear' 'stormy']"] = "stormy"
df_joined["general_w"][df_joined["general_w"] == "['cloudy' 'foggy']"] = "foggy"
df_joined["general_w"][df_joined["general_w"] == "['cloudy' 'rainy' 'stormy']"] = "stormy"
df_joined["general_w"].unique()
# for the moment I just fill NaNs with previous value. 
df_joined["general_w"].fillna(method="ffill", inplace=True)
# then dummies for general weather
dummies = pd.get_dummies(df_joined['general_w'], prefix="general_w")
df_joined = df_joined.join(dummies)

# column for weekend or not
df_joined["weekend"] = df_joined.day_of_week.isin([5, 6]).astype("float")

# create dummies for school holidays
dummies = pd.get_dummies(df_joined['s_hol_name'], prefix="sh")
df_joined = df_joined.join(dummies)

# and dummies for public holidays
dummies = pd.get_dummies(df_joined['p_hol_name'], prefix="ph")
df_joined = df_joined.join(dummies)

# get temperature change from one to another day
tmp_temp = df_joined.temp_mean.copy()
tmp_temp_day_before = np.array(tmp_temp[15:]) - np.array(tmp_temp[:-15])
tmp_temp = list(chain.from_iterable([list(np.repeat("NaN",15)), list(tmp_temp_day_before)]))
df_joined["temp_shift"] = tmp_temp

# get pressure change as proxy for change of weather
tmp_pr = df_joined.pressure_mean.copy()
tmp_pr_day_before = np.array(tmp_pr[15:]) - np.array(tmp_pr[:-15])
tmp_pr = list(chain.from_iterable([list(np.repeat("NaN",15)), list(tmp_pr_day_before)]))
df_joined["pressure_shift"] = tmp_pr

# tomorrows weather
tmp_weather = df_joined["general_w"].copy()
tmp_next_day = list(chain.from_iterable([list(tmp_weather[15:]), list(np.repeat("NaN",15))]))
df_joined["tomorrows_weather"] = tmp_next_day

# then dummies for tomorrows weather
dummies = pd.get_dummies(df_joined['tomorrows_weather'], prefix="tw")
df_joined = df_joined.join(dummies)

In [ ]:
# export combined data to csv file
df_joined.to_csv('../data/data_combined.csv', index=False)